In [3]:
import guardrails as gd
from pydantic import BaseModel, Field
from typing import ClassVar, Dict, Any
from guardrails.hub import GuardrailsPII 
import json # Keep this import, we will use it explicitly for dumps

# --- 1. Define the Structured Output with PII Protection ---

class LLMAnswer(BaseModel):
    """
    Pydantic Model defining the LLM's expected output structure.
    """
    
    # Define pii_validator as a ClassVar to prevent Pydantic from treating it as a data field.
    pii_validator: ClassVar[GuardrailsPII] = GuardrailsPII(
        entities=["EMAIL_ADDRESS", "PHONE_NUMBER", "US_SSN"], 
        on_fail="fix" 
    )

    response: str = Field(
        description="The LLM's full answer to the question.",
        validators=[pii_validator]
    )
    
    context_source: str = Field(
        description="The source document or context used for the answer."
    )


# --- 2. Simulate the LLM's Behavior (Raw Output) ---

def generate_unsafe_llm_response() -> str:
    """
    Simulates a Large Language Model returning raw JSON output that includes PII.
    """
    raw_response_data = {
        "response": "The new policy details were sent to John Doe at john.doe@secure-corp.net. His mobile number is 555-867-5309. This information is confidential.",
        "context_source": "Policy Manual V1.2"
    }
    return json.dumps(raw_response_data)


# --- 3. Execute the Guarded Flow (FIXED) ---

def run_pii_redaction_test():
    """
    Initializes the Guard from the Pydantic model and runs the test, 
    using modern Pydantic V2 methods.
    """
    n
    guard = gd.Guard.from_pydantic(output_class=LLMAnswer)
    raw_llm_output = generate_unsafe_llm_response()
    
    print("="*80)
    print("🔴 RAW LLM OUTPUT (Contains PII)")
    print("="*80)
    print(raw_llm_output)
    
    print("\n" + "="*80)
    print("🟡 GUARDRAIL PARSING & REDACTION")
    print("="*80)
    
    try:
        # 1. Get the ValidationOutcome object. PII detection and redaction happens here.
        outcome = guard.parse(llm_output=raw_llm_output)

        # 2. Get the validated output, which is a Python dictionary after fixing.
        validated_output_data: Dict[str, Any] = outcome.validated_output

        # 3. Convert the dictionary back to a JSON string.
        #    This is safe and avoids the internal Guardrails issue.
        validated_json_string = json.dumps(validated_output_data)
        
        # 4. ✅ Pydantic V2 FIX: Use model_validate_json instead of parse_raw
        final_output = LLMAnswer.model_validate_json(validated_json_string)

        print("\n" + "="*80)
        print("✅ GUARDED FINAL OUTPUT (PII Redacted)")
        print("="*80)
        
        # 5. ✅ Pydantic V2 FIX: Use model_dump_json instead of json()
        #    Use indent=4 to format the output nicely.
        print(final_output.model_dump_json(indent=4))
    

    except Exception as e:
        print(f"An error occurred during guardrail processing: {e}")


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

c:\venv311\Lib\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
C:\Users\adminuser\AppData\Local\Temp\ipykernel_6236\3134622074.py:20: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'validators'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  response: str = Field(


In [2]:
run_pii_redaction_test()

c:\venv311\Lib\site-packages\guardrails\validator_service\__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🔴 RAW LLM OUTPUT (Contains PII)
{"response": "The new policy details were sent to John Doe at john.doe@secure-corp.net. His mobile number is 555-867-5309. This information is confidential.", "context_source": "Policy Manual V1.2"}

🟡 GUARDRAIL PARSING & REDACTION

✅ GUARDED FINAL OUTPUT (PII Redacted)
{
    "response": "The new policy details were sent to John Doe at <EMAIL_ADDRESS>. His mobile number is <PHONE_NUMBER>. This information is confidential.",
    "context_source": "Policy Manual V1.2"
}

--- PII Summary ---
Original Email: john.doe@secure-corp.net  -> Redacted to [EMAIL_ADDRESS]
Original Phone: 555-867-5309             -> Redacted to [PHONE_NUMBER]


In [3]:
run_pii_redaction_test()

c:\venv311\Lib\site-packages\guardrails\validator_service\__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🔴 RAW LLM OUTPUT (Contains PII)
{"response": "The new policy details were sent to John Doe at john.doe@secure-corp.net. His mobile number is 555-867-5309. This information is confidential.", "context_source": "Policy Manual V1.2"}

✅ GUARDED FINAL OUTPUT (PII Redacted)
An error occurred during guardrail processing: 1 validation error for LLMAnswer
__root__
  the JSON object must be str, bytes or bytearray, not ValidationOutcome [type=type_error, input_value=ValidationOutcome(call_id...passed=True, error=None), input_type=ValidationOutcome]


C:\Users\adminuser\AppData\Local\Temp\ipykernel_1224\1345588614.py:72: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  final_output = LLMAnswer.parse_raw(validated_output_json_string)


In [10]:
input_ban_validator.validate("athena")

FailResult(outcome='fail', error_message='Output contains banned words', fix_value='a', error_spans=[ErrorSpan(start=1, end=6, reason="Found match with banned word 'athena' in 'thena'")], metadata=None, validated_chunk=None)

In [14]:
input_ban_validator.validate("athena")

FailResult(outcome='fail', error_message='Output contains banned words', fix_value='a', error_spans=[ErrorSpan(start=1, end=6, reason="Found match with banned word 'athena' in 'thena'")], metadata=None, validated_chunk=None)

In [4]:
import guardrails as gd
from pydantic import BaseModel, Field
from typing import ClassVar, Dict, Any
import json
from guardrails.hub import GuardrailsPII 
from guardrails.hub import BanList 
# ✅ FINAL FIX: Import GuardrailsError directly from the top-level 'guardrails' package
# If this fails, the except block will still catch the base Exception.
try:
    from guardrails.errors import GuardrailsError
except ImportError:
    # Fallback to catching the base Python Exception if the library hides its own error class
    GuardrailsError = Exception

# --- 1. Define Output Guardrail (PII Redaction) ---

class LLMAnswer(BaseModel):
    """
    Pydantic Model for the safe, structured LLM output.
    """
    
    # OUTPUT VALIDATOR (GuardrailsPII)
    pii_validator: ClassVar[GuardrailsPII] = GuardrailsPII(
        entities=["EMAIL_ADDRESS", "PHONE_NUMBER", "US_SSN"], 
        # Action: Automatically redact PII
        on_fail="fix" 
    )

    response: str = Field(
        description="The LLM's full answer to the question.",
        validators=[pii_validator]
    )
    
    context_source: str = Field(
        description="The source document or context used for the answer."
    )

# --- 2. Define Input Guardrail (BanList for Sensitive Terms) ---

SENSITIVE_TERMS = ["Atin", "codename", "project-x", "athena", "launch-date"]

# The BanList validator is configured to raise an exception if a banned word is found.
input_ban_validator = BanList(
    banned_words=SENSITIVE_TERMS,
    on_fail="exception" 
)


# --- 3. Simulate the LLM's Behavior (for Output Guardrail Test) ---

def generate_unsafe_llm_response() -> str:
    """
    Simulates a RAW response containing PII.
    """
    raw_response_data = {
        "response": "The new policy details were sent to John Doe at john.doe@secure-corp.net. His mobile number is 555-867-5309. This information is confidential.",
        "context_source": "Policy Manual V1.2"
    }
    return json.dumps(raw_response_data)


# --- 4. Execute the Full Guarded Flow ---

def run_guarded_flow(user_prompt: str, raw_llm_output: str):
    """
    Runs the full flow: Input Check -> Output Fix.
    """
    
    print("="*80)
    print(f"User Prompt Received: {user_prompt}")
    print("="*80)
    
    # --- INPUT GUARDRAIL CHECK (BanList) ---
    try:
        print("🟡 STEP 1: Applying Input Guardrail (BanList)...")

        SENSITIVE_TERMS = ["Atin", "codename", "project-x", "athena", "launch-date"]

        # The BanList validator is configured to raise an exception if a banned word is found.
        input_ban_validator = BanList(
            banned_words=SENSITIVE_TERMS,
            on_fail="exception" 
        )

        # We use the validator's validate() method directly on the simple string input.
        # This will raise GuardrailsError if a banned word is detected.
        print(f"Starting validation on {user_prompt}")
        input_ban_validator.validate(user_prompt)
        print(f"Done validation on {user_prompt}")
        
        print("✅ Input Guardrail: Prompt is safe and compliant (no banned terms detected).")
        
    except Exception as e:
        print(f"\n❌ Input Guardrail failed due to an unexpected error: {e}")
        return


    # --- OUTPUT GUARDRAIL EXECUTION (GuardrailsPII) ---
    print("\n" + "="*80)
    print("🔴 RAW LLM Output for Testing:")
    print("="*80)
    print(raw_llm_output)
    
    print("\n🟡 STEP 2: Applying Output Guardrail (PII Redaction)...")
    
    try:
        output_guard = gd.Guard.from_pydantic(output_class=LLMAnswer)

        # PII detection and redaction happens here.
        outcome = output_guard.parse(llm_output=raw_llm_output)

        validated_output_data: Dict[str, Any] = outcome.validated_output
        validated_json_string = json.dumps(validated_output_data)
        
        # Parse the safe JSON string into the final Pydantic object
        final_output = LLMAnswer.model_validate_json(validated_json_string)

        print("\n✅ Output Guardrail: PII Redacted Successfully.")
        print("--- FINAL SAFE RESPONSE ---")
        
        # Display the safe, structured data
        print(final_output.model_dump_json(indent=4))
        
        print("\n--- PII Summary ---")
        print("Redaction applied to email and phone number.")

    except Exception as e:
        print(f"An error occurred during guardrail processing: {e}")


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

c:\venv311\Lib\site-packages\transformers\convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
C:\Users\adminuser\AppData\Local\Temp\ipykernel_12560\3095178899.py:29: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'validators'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  response: str = Field(


In [5]:
# --- Test 1: Safe Input (Flow should complete and redact PII) ---
safe_input = "Can you summarize the policy details and who the contact person is?"
run_guarded_flow(safe_input, generate_unsafe_llm_response())

print("\n" + "#"*80 + "\n")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


User Prompt Received: Can you summarize the policy details and who the contact person is?
🟡 STEP 1: Applying Input Guardrail (BanList)...
Starting validation on Can you summarize the policy details and who the contact person is?
Done validation on Can you summarize the policy details and who the contact person is?
✅ Input Guardrail: Prompt is safe and compliant (no banned terms detected).

🔴 RAW LLM Output for Testing:
{"response": "The new policy details were sent to John Doe at john.doe@secure-corp.net. His mobile number is 555-867-5309. This information is confidential.", "context_source": "Policy Manual V1.2"}

🟡 STEP 2: Applying Output Guardrail (PII Redaction)...

✅ Output Guardrail: PII Redacted Successfully.
--- FINAL SAFE RESPONSE ---
{
    "response": "The new policy details were sent to John Doe at <EMAIL_ADDRESS>. His mobile number is <PHONE_NUMBER>. This information is confidential.",
    "context_source": "Policy Manual V1.2"
}

--- PII Summary ---
Redaction applied to em

In [7]:
# --- Test 2: Unsafe Input (Fuzzy match test - Flow should fail at Step 1) ---
# The term 'athena' is banned. 'a t h e n a' or 'atena' would also fail due to fuzzy search.
unsafe_input = "athena"
run_guarded_flow(unsafe_input, generate_unsafe_llm_response())

User Prompt Received: athena
🟡 STEP 1: Applying Input Guardrail (BanList)...
Starting validation on athena
Done validation on athena
✅ Input Guardrail: Prompt is safe and compliant (no banned terms detected).

🔴 RAW LLM Output for Testing:
{"response": "The new policy details were sent to John Doe at john.doe@secure-corp.net. His mobile number is 555-867-5309. This information is confidential.", "context_source": "Policy Manual V1.2"}

🟡 STEP 2: Applying Output Guardrail (PII Redaction)...

✅ Output Guardrail: PII Redacted Successfully.
--- FINAL SAFE RESPONSE ---
{
    "response": "The new policy details were sent to John Doe at <EMAIL_ADDRESS>. His mobile number is <PHONE_NUMBER>. This information is confidential.",
    "context_source": "Policy Manual V1.2"
}

--- PII Summary ---
Redaction applied to email and phone number.
